In [112]:
import matplotlib.pyplot as plt
import numpy as np
import pyxdf
import mne
import pandas as pd

from matplotlib.colors import TwoSlopeNorm
import seaborn as sns
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import tfr_multitaper
from mne.stats import permutation_cluster_1samp_test as pcluster_test

%matplotlib widget

In [123]:
def get_index_of_stream_type(streams, type):
    """
    type = 'EEG', 'TargetID'
    return the index of the stream corresponding to the wanted type
    types : 'EEG', 'Markers', 'Unity.Property', 'Unity.Marker'(2 different),
    """
    if type == 'EEG': name = 'eegoSports 000'
    
    elif type == 'TargetID': name = 'MI trial marker'
    else :
        raise NameError(
            f"given type '{type}' is not an argument. Accepted types are 'EEG' and 'TargetID'"
        )
    for i in range(len(streams)):
        if streams[i]['info']['name'][0][:14] == name:
            return i

    raise NameError(
         f"given type '{type}' cannot be found in any of the streams"
    )

In [124]:
def get_raw_data(EEG):
    data = streams[EEG]["time_series"].copy().T  * 1e-6
    sfreq = float(streams[EEG]["info"]["nominal_srate"][0])
    ch_types = ['eeg'] * 64 + ['misc'] + ['bio']
    info_passive = mne.create_info(channel_label, sfreq, ch_types)
    raw_passive = mne.io.RawArray(data, info_passive) 
    print("\n  \n  Raw data information \n \n ", raw_passive.info)
    raw_passive.info['bads'].extend(['M1', 'M2'])
    raw_passive.plot(scalings=0.00013); 
    return raw_passive

In [125]:
def preproc_eeg(raw_passive, freq_low, freq_high, car = True):
    raw_passive_filtered = raw_passive.filter(l_freq=freq_low, h_freq=freq_high)
    if car:
        raw_passive_filtered, data_ref = mne.set_eeg_reference(raw_passive_filtered, ref_channels = 'average', copy = True)
    raw_passive_filtered.plot();
    print("\n \n  Filtered data information \n \n ", raw_passive_filtered.info)
    return raw_passive_filtered

In [126]:
def get_epochs(streams, EEG, tiral_id, filtered_data, time_offset, duration):
    reset_time = streams[EEG]['time_stamps'][0]
    trial_time = streams[trial_id]['time_stamps'] - reset_time
    targets = np.array(streams[trial_id]['time_series']).squeeze()
    annotations = mne.Annotations(onset = trial_time, duration = 0., description = targets)
    data_annotated = filtered_data.copy().set_annotations(annotations)
    events_p01, event_id_p01 = mne.events_from_annotations(data_annotated)
    epochs_p01 = mne.Epochs(filtered_data, events_p01, event_id_p01, tmin=-time_offset, tmax = duration, baseline = (-time_offset,0), reject=dict(eeg=400e-6), preload=True)
    epochs_p01['2'].plot(events=events_p01, event_id=event_id_p01); 
    epochs_p01['5'].plot(events=events_p01, event_id=event_id_p01); 
    epochs_p01['8'].plot(events=events_p01, event_id=event_id_p01); 

In [127]:
GROUP_REALISTIC_ARM = 'DATA/Group_Realistic_Arm/'
SUBJECT = 'S01/'
BASELINE_EO = 'sub-S01_eegBaselineOE.xdf'
BASELINE_EC = 'sub-S01_eegBaselineCE.xdf'
MI_PRE = 'sub-S01_MIpre.xdf'
PO_BLOCK1 = 'sub_S01_PO_Block1.xdf'
PO_BLOCK2 = 'sub_S01_PO_Block2.xdf'
MI_POST = 'sub_S01_MIpost.xdf'
EEG_MONTAGE = 'DATA/montage_perfect.csv'
bandpass_cutoff_low = 1.5
bandpass_cutoff_high = 40
time_offset = 1
duration = 5

In [128]:
file_name = GROUP_REALISTIC_ARM+SUBJECT+MI_PRE

streams, header = pyxdf.load_xdf(file_name)
EEG = get_index_of_stream_type(streams, type='EEG')
trial_id = get_index_of_stream_type(streams, 'TargetID')

montage_label = pd.read_csv(EEG_MONTAGE)
channel_label = montage_label['ch_name'].tolist()
channel_label.append('Trigger')
channel_label.append('GSR')

raw_data = get_raw_data(EEG, EEG_MONTAGE)

filtered_data = preproc_eeg(raw_data, bandpass_cutoff_low, bandpass_cutoff_high)

epochs = get_epochs(streams, EEG, trial_id, filtered_data, time_offset, duration)


Stream 2: Calculated effective sampling rate 15487.4682 Hz is different from specified rate 500.0000 Hz.
Stream 1: Calculated effective sampling rate 5192.4444 Hz is different from specified rate 500.0000 Hz.


NameError: given type 'TargetID' cannot be found in any of the streams